In [2]:
import pandas as pd
import tushare as ts
import numpy as np

In [3]:
pro = ts.pro_api()

In [4]:
#从tushare读取数据

df_0828 = pro.daily(trade_date='20230828')
df_0829 = pro.daily(trade_date='20230829')
df_0830 = pro.daily(trade_date='20230830')
df_0831 = pro.daily(trade_date='20230831')
df_0901 = pro.daily(trade_date='20230901')

df_0904 = pro.daily(trade_date='20230904')
df_0905 = pro.daily(trade_date='20230905')
df_0906 = pro.daily(trade_date='20230906')
df_0907 = pro.daily(trade_date='20230907')
df_0908 = pro.daily(trade_date='20230908')

df = pd.concat([df_0828,df_0829,df_0830,df_0831,df_0901,df_0904,df_0905,df_0906,df_0907,df_0908],axis=0)

In [6]:
        suspend_0828 = pro.query('suspend', ts_code='', suspend_date='20230828', resume_date='', fields='')
        resume_0828 = pro.query('suspend', ts_code='', suspend_date='', resume_date='20230828', fields='')
        
        suspend_0829 = pro.query('suspend', ts_code='', suspend_date='20230829', resume_date='', fields='')
        resume_0829 = pro.query('suspend', ts_code='', suspend_date='', resume_date='20230829', fields='')
        
        suspend_0830 = pro.query('suspend', ts_code='', suspend_date='20230830', resume_date='', fields='')
        resume_0830 = pro.query('suspend', ts_code='', suspend_date='', resume_date='20230830', fields='')
        
        suspend_0831 = pro.query('suspend', ts_code='', suspend_date='20230831', resume_date='', fields='')
        resume_0831 = pro.query('suspend', ts_code='', suspend_date='', resume_date='20230831', fields='')
        
        suspend_0901 = pro.query('suspend', ts_code='', suspend_date='20230901', resume_date='', fields='')
        resume_0901 = pro.query('suspend', ts_code='', suspend_date='', resume_date='20230901', fields='')
        
        suspend_0904 = pro.query('suspend', ts_code='', suspend_date='20230904', resume_date='', fields='')
        resume_0904 = pro.query('suspend', ts_code='', suspend_date='', resume_date='20230904', fields='')
        
        suspend_0905 = pro.query('suspend', ts_code='', suspend_date='20230905', resume_date='', fields='')
        resume_0905 = pro.query('suspend', ts_code='', suspend_date='', resume_date='20230905', fields='')
        
        suspend_0906 = pro.query('suspend', ts_code='', suspend_date='20230906', resume_date='', fields='')
        resume_0906 = pro.query('suspend', ts_code='', suspend_date='', resume_date='20230906', fields='')
        
        suspend_0907 = pro.query('suspend', ts_code='', suspend_date='20230907', resume_date='', fields='')
        resume_0907 = pro.query('suspend', ts_code='', suspend_date='', resume_date='20230907', fields='')
        
        suspend_0908 = pro.query('suspend', ts_code='', suspend_date='20230908', resume_date='', fields='')
        resume_0908 = pro.query('suspend', ts_code='', suspend_date='', resume_date='20230908', fields='')
        
        suspend = pd.concat([suspend_0828,suspend_0829,suspend_0830,suspend_0831,suspend_0901,suspend_0904,suspend_0905,suspend_0906,suspend_0907,suspend_0908])[['ts_code','suspend_date']]
        resume = pd.concat([resume_0828,resume_0829,resume_0830,resume_0831,resume_0901,resume_0904,resume_0905,resume_0906,resume_0907,resume_0908])[['ts_code','resume_date']]


In [9]:
suspend['suspend_type'] = 'S'
resume['suspend_type'] = 'R'

suspend.rename(columns={'suspend_date': 'date','ts_code':'asset'}, inplace=True)
resume.rename(columns={'resume_date': 'date','ts_code':'asset'}, inplace=True)

suspend_df = pd.concat([suspend,resume],ignore_index=True)

In [6]:
#修改股票代码格式
ls = []
ts_code = []
for i in suspend_df['asset'].values:
    ls = i.split('.')
    if ls[1] == 'SZ':
        ls[1] = 'XSHE'
    elif ls[1] == 'SH':
        ls[1] = 'XSHG'
    else:
        ls[1] = ''
    ts_code.append(ls[0]+'.'+ls[1])

suspend_df['asset'] = ts_code
suspend_df['date'] = suspend_df['date'].astype('int64')

In [7]:
#修改股票代码格式
ls = []
ts_code = []
for i in df['ts_code'].values:
    ls = i.split('.')
    if ls[1] == 'SZ':
        ls[1] = 'XSHE'
    else:
        ls[1] = 'XSHG'
    ts_code.append(ls[0]+'.'+ls[1])

df['ts_code'] = ts_code
df.rename(columns={'trade_date': 'date','ts_code':'asset'}, inplace=True)

In [8]:
#将两个数据的date转换为同一格式
df['date'] = df['date'].astype('int64')

In [9]:
#取出要用的列
df = df[['asset','date','close','pre_close']]

In [10]:
#读取权重数据
weight_data = pd.read_csv('/Users/eren/Desktop/Code/0904data.csv')

In [11]:
#合并数据，使权重与股票信息合并
dff = df.merge(weight_data, on=['date','asset'], how='right')

In [12]:
dff = dff.merge(suspend_df, on=['date','asset'], how='left')

In [13]:
dff.drop(dff[dff['weight'] == 0].index, inplace=True)


# 股票停牌、退市处理
groups = dff.groupby('asset')
groups_ready = []
for stock_key, stock_data in groups:
    # 初始时间段，股票停牌处理
    stock_data = stock_data.reset_index(drop=True)
    first_valid_index = stock_data['close'].first_valid_index()
    stock_data.drop(stock_data[stock_data.index < first_valid_index].index, inplace=True)
    stock_data = stock_data.reset_index(drop=True)

    # 股票退市处理
    close_last_valid_index = stock_data['close'].last_valid_index()
    suspend_type_last_valid_index = stock_data['suspend_type'].last_valid_index()
    if suspend_type_last_valid_index:
        if suspend_type_last_valid_index > close_last_valid_index:
            last_index = suspend_type_last_valid_index + 1
            stock_data.drop(stock_data[stock_data.index > last_index].index, inplace=True)
            stock_data.loc[last_index, 'pre_close'] = 1
            stock_data.loc[last_index, 'close'] = 0

    stock_data.drop('suspend_type', axis=1, inplace=True)
    groups_ready.append(stock_data)

# 合并时处理所有待处理的停牌股票
right_data = pd.concat(groups_ready).sort_values(by='date', ascending=True).fillna(1).reset_index(drop=True)


# 再次权重归一化
to_normalize_groups = right_data.groupby('date')
normalized_groups = []
for name, group in to_normalize_groups:
    group.loc[group['weight'] < 0, 'weight'] = 0
    group['weight'] = group['weight'] / group['weight'].sum()
    # 调整，以确保权重和在指定精度范围内
    while True:
        sum_normalized = group['weight'].sum()
        if sum_normalized < 0.99999:
            adjustment_factor = 0.99999 / sum_normalized
            group['weight'] *= adjustment_factor
        elif sum_normalized > 1.00001:
            adjustment_factor = 1.00001 / sum_normalized
            group['weight'] *= adjustment_factor
        else:
            break

    normalized_groups.append(group)
normalized_index = pd.concat(normalized_groups)

# 创造pre_weight列
normalized_index['pre_weight'] = normalized_index.groupby('asset')['weight'].shift(1)
normalized_index.fillna(0,inplace=True)

In [14]:
normalized_index

,asset,date,close,pre_close,weight,pre_weight
0,000002.XSHE,20230904,14.04,14.05,0.000409,0.000000
1,603180.XSHG,20230904,32.08,29.16,0.000204,0.000000
2,002456.XSHE,20230904,5.87,5.70,0.000613,0.000000
3,300862.XSHE,20230904,25.32,25.03,0.000618,0.000000
4,688163.XSHG,20230904,19.70,19.80,0.000353,0.000000
...,...,...,...,...,...,...
13108,300543.XSHE,20230908,10.08,10.04,0.000419,0.000419
13109,300538.XSHE,20230908,19.45,20.76,0.000059,0.000059
13110,603506.XSHG,20230908,12.62,12.89,0.000754,0.000754
13111,300552.XSHE,20230908,20.06,19.40,0.000184,0.000184


In [15]:
#计算累计收益率
R = 1
per = 0.0015
all = []
for date,data in normalized_index.groupby('date'):
    r = sum(((data['close']-data['pre_close'])/data['pre_close'])*data['weight'])
    commission = sum(np.abs(data['weight']-data['pre_weight'])*per*2)
    r -= commission
    R = R*(1+r)
    all.append(R)
print(all)

[1.0100394598325548, 1.0074983758215352, 1.0145221003151046, 0.9977342099113076, 1.0020234011760996]


In [16]:
#获取股票基本信息（所属行业）  （一小时仅可获取一次）
stock_info = pro.stock_basic(list_status='L', fields='ts_code,name,industry')

print(stock_info)

        ts_code     name industry
0     000001.SZ     平安银行       银行
1     000002.SZ      万科A     全国地产
2     000004.SZ     国华网安     软件服务
3     000005.SZ     ST星源     环境保护
4     000006.SZ     深振业A     区域地产
...         ...      ...      ...
5269  873339.BJ     恒太照明      元器件
5270  873527.BJ      夜光明     化工原料
5271  873576.BJ     天力复合      小金属
5272  873593.BJ     鼎智科技     机械基件
5273  689009.SH  九号公司-WD      摩托车

[5274 rows x 3 columns]


In [17]:
#为获取的数据修改股票名称
ls = []
ts_code = []
for i in stock_info['ts_code'].values:
    ls = i.split('.')
    if ls[1] == 'SZ':
        ls[1] = 'XSHE'
    elif ls[1] == 'SH':
        ls[1] = 'XSHG'
    else:
        ls[1] = ''
    ts_code.append(ls[0]+'.'+ls[1])

stock_info['ts_code'] = ts_code

stock_info.rename(columns={'ts_code':'asset'}, inplace=True)
stock_info.drop(columns='name', inplace=True)

In [18]:
#合并权重数据和行业数据
s = dff.merge(stock_info, on='asset', how='left')

In [19]:
s.groupby(['date','industry'])['weight'].sum()

date      industry
20230904  汽车配件        0.000013
20230905  元器件         0.001443
          文教休闲        0.000100
Name: weight, dtype: float64

In [20]:
data_dic = s[s['date'] == s.groupby('date').last().index[-1]].groupby('industry')['weight'].sum().sort_values().to_dict()

In [21]:
sorted_data = dict(sorted(data_dic.items(), key=lambda item: item[1], reverse=True))

In [22]:
weight_X = list(sorted_data.keys())
weight_y = list(sorted_data.values())

In [23]:
pie_data = []
for index, value in sorted_data.items():
    pie_data.append({"value": value, "name": index})
    if len(pie_data) == 10:
        break

In [24]:
#需要输出的累积收益数据
time_Y = all
time_x = list(dff['date'].unique())